# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f01acf0eee0>
├── 'a' --> tensor([[-2.1893,  0.5867, -0.9865],
│                   [-0.5054,  0.0042,  2.1046]])
└── 'x' --> <FastTreeValue 0x7f01acf0e070>
    └── 'c' --> tensor([[ 0.7638,  1.0356, -0.2120, -0.0864],
                        [-0.6051,  0.8182, -1.8605,  1.0783],
                        [ 0.4547, -1.8641,  0.5639, -0.7374]])

In [4]:
t.a

tensor([[-2.1893,  0.5867, -0.9865],
        [-0.5054,  0.0042,  2.1046]])

In [5]:
%timeit t.a

67 ns ± 0.0275 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f01acf0eee0>
├── 'a' --> tensor([[ 0.5785,  0.2696,  2.2569],
│                   [ 0.4889, -0.5327, -0.4742]])
└── 'x' --> <FastTreeValue 0x7f01acf0e070>
    └── 'c' --> tensor([[ 0.7638,  1.0356, -0.2120, -0.0864],
                        [-0.6051,  0.8182, -1.8605,  1.0783],
                        [ 0.4547, -1.8641,  0.5639, -0.7374]])

In [7]:
%timeit t.a = new_value

67.9 ns ± 0.0291 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-2.1893,  0.5867, -0.9865],
               [-0.5054,  0.0042,  2.1046]]),
    x: Batch(
           c: tensor([[ 0.7638,  1.0356, -0.2120, -0.0864],
                      [-0.6051,  0.8182, -1.8605,  1.0783],
                      [ 0.4547, -1.8641,  0.5639, -0.7374]]),
       ),
)

In [10]:
b.a

tensor([[-2.1893,  0.5867, -0.9865],
        [-0.5054,  0.0042,  2.1046]])

In [11]:
%timeit b.a

52.6 ns ± 0.038 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0972,  1.4967, -0.6090],
               [-1.1739, -1.0954,  1.8008]]),
    x: Batch(
           c: tensor([[ 0.7638,  1.0356, -0.2120, -0.0864],
                      [-0.6051,  0.8182, -1.8605,  1.0783],
                      [ 0.4547, -1.8641,  0.5639, -0.7374]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.169 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

827 ns ± 0.0768 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 29.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 477 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 353 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f00f616db50>
├── 'a' --> tensor([[[-2.1893,  0.5867, -0.9865],
│                    [-0.5054,  0.0042,  2.1046]],
│           
│                   [[-2.1893,  0.5867, -0.9865],
│                    [-0.5054,  0.0042,  2.1046]],
│           
│                   [[-2.1893,  0.5867, -0.9865],
│                    [-0.5054,  0.0042,  2.1046]],
│           
│                   [[-2.1893,  0.5867, -0.9865],
│                    [-0.5054,  0.0042,  2.1046]],
│           
│                   [[-2.1893,  0.5867, -0.9865],
│                    [-0.5054,  0.0042,  2.1046]],
│           
│                   [[-2.1893,  0.5867, -0.9865],
│                    [-0.5054,  0.0042,  2.1046]],
│           
│                   [[-2.1893,  0.5867, -0.9865],
│                    [-0.5054,  0.0042,  2.1046]],
│           
│                   [[-2.1893,  0.5867, -0.9865],
│                    [-0.5054,  0.0042,  2.1046]]])
└── 'x' --> <FastTreeValue 0x7f00f616d280>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 25.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f00f616d8b0>
├── 'a' --> tensor([[-2.1893,  0.5867, -0.9865],
│                   [-0.5054,  0.0042,  2.1046],
│                   [-2.1893,  0.5867, -0.9865],
│                   [-0.5054,  0.0042,  2.1046],
│                   [-2.1893,  0.5867, -0.9865],
│                   [-0.5054,  0.0042,  2.1046],
│                   [-2.1893,  0.5867, -0.9865],
│                   [-0.5054,  0.0042,  2.1046],
│                   [-2.1893,  0.5867, -0.9865],
│                   [-0.5054,  0.0042,  2.1046],
│                   [-2.1893,  0.5867, -0.9865],
│                   [-0.5054,  0.0042,  2.1046],
│                   [-2.1893,  0.5867, -0.9865],
│                   [-0.5054,  0.0042,  2.1046],
│                   [-2.1893,  0.5867, -0.9865],
│                   [-0.5054,  0.0042,  2.1046]])
└── 'x' --> <FastTreeValue 0x7f00f616db80>
    └── 'c' --> tensor([[ 0.7638,  1.0356, -0.2120, -0.0864],
                        [-0.6051,  0.8182, -1.8605,  1.0783],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 28.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.2 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-2.1893,  0.5867, -0.9865],
                [-0.5054,  0.0042,  2.1046]],
       
               [[-2.1893,  0.5867, -0.9865],
                [-0.5054,  0.0042,  2.1046]],
       
               [[-2.1893,  0.5867, -0.9865],
                [-0.5054,  0.0042,  2.1046]],
       
               [[-2.1893,  0.5867, -0.9865],
                [-0.5054,  0.0042,  2.1046]],
       
               [[-2.1893,  0.5867, -0.9865],
                [-0.5054,  0.0042,  2.1046]],
       
               [[-2.1893,  0.5867, -0.9865],
                [-0.5054,  0.0042,  2.1046]],
       
               [[-2.1893,  0.5867, -0.9865],
                [-0.5054,  0.0042,  2.1046]],
       
               [[-2.1893,  0.5867, -0.9865],
                [-0.5054,  0.0042,  2.1046]]]),
    x: Batch(
           c: tensor([[[ 0.7638,  1.0356, -0.2120, -0.0864],
                       [-0.6051,  0.8182, -1.8605,  1.0783],
                       [ 0.4547, -1.8641,  0.5639, -0.7374]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 194 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-2.1893,  0.5867, -0.9865],
               [-0.5054,  0.0042,  2.1046],
               [-2.1893,  0.5867, -0.9865],
               [-0.5054,  0.0042,  2.1046],
               [-2.1893,  0.5867, -0.9865],
               [-0.5054,  0.0042,  2.1046],
               [-2.1893,  0.5867, -0.9865],
               [-0.5054,  0.0042,  2.1046],
               [-2.1893,  0.5867, -0.9865],
               [-0.5054,  0.0042,  2.1046],
               [-2.1893,  0.5867, -0.9865],
               [-0.5054,  0.0042,  2.1046],
               [-2.1893,  0.5867, -0.9865],
               [-0.5054,  0.0042,  2.1046],
               [-2.1893,  0.5867, -0.9865],
               [-0.5054,  0.0042,  2.1046]]),
    x: Batch(
           c: tensor([[ 0.7638,  1.0356, -0.2120, -0.0864],
                      [-0.6051,  0.8182, -1.8605,  1.0783],
                      [ 0.4547, -1.8641,  0.5639, -0.7374],
                      [ 0.7638,  1.0356, -0.2120, -0.0864],
                      [-0.6051,  

In [28]:
%timeit Batch.cat(batches)

145 µs ± 235 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

301 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
